In [1]:
import turicreate as tc


In [2]:
tc.config.set_num_gpus(1)

In [3]:
# Load the date
data =  tc.SFrame("./ig02.sframe/")

# Make a train-test split
train_data, test_data = data.random_split(0.8)

In [4]:
# Create a model
model = tc.object_detector.create(train_data)

# Save predictions to an SArray
predictions = model.predict(test_data)

# Evaluate the model and save the results into a dictionary
metrics = model.evaluate(test_data)

Using 'image' as feature column
Using 'annotations' as annotations column
Setting 'batch_size' to 32
Using GPU to create model (GeForce GTX 1060)
Setting 'max_iterations' to 5000
+--------------+--------------+--------------+
| Iteration    | Loss         | Elapsed Time |
+--------------+--------------+--------------+
| 1            | 6.198        | 8.5          |
| 12           | 6.107        | 19.0         |
| 23           | 5.807        | 29.9         |
| 34           | 5.131        | 40.3         |
| 44           | 4.550        | 50.4         |
| 55           | 4.302        | 61.2         |
| 66           | 4.145        | 72.0         |
| 77           | 3.875        | 82.8         |
| 88           | 3.791        | 93.7         |
| 99           | 3.547        | 104.7        |
| 110          | 3.415        | 115.3        |
| 121          | 3.333        | 125.8        |
| 133          | 3.277        | 135.9        |
| 145          | 3.234        | 146.5        |
| 157          | 3.215

In [5]:
# Save the model for later use in Turi Create
model.save('mymodel.model')

# Export for use in Core ML
model.export_coreml('MyCustomObjectDetector.mlmodel')